## Loading Data

In [79]:
from sklearn.datasets import load_iris
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load the iris dataset
iris = load_iris()
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris['target'] = iris.target

# Create features and target numpy arrays
features = df_iris[iris.feature_names]
X = df_iris[iris.feature_names].values
target = df_iris['target']
y = df_iris['target'].values

print(features.head(6))
# print(X)
print(target.head(6))
# print(y)

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2
5                5.4               3.9                1.7               0.4
0    0
1    0
2    0
3    0
4    0
5    0
Name: target, dtype: int64


In [80]:
import torch
from torch.utils.data import TensorDataset

dataset = TensorDataset(torch.tensor(X), torch.tensor(y))
dataset
type(dataset)




torch.utils.data.dataset.TensorDataset

In [81]:
a, b = dataset[0]
print(dataset[0])
print(a)
print(b)


(tensor([5.1000, 3.5000, 1.4000, 0.2000], dtype=torch.float64), tensor(0))
tensor([5.1000, 3.5000, 1.4000, 0.2000], dtype=torch.float64)
tensor(0)


In [82]:
from torch.utils.data import DataLoader

batch_size = 2 # determine how many samples are included in each iteration (efficient training)
shuffle = True # randomize training data at each epoch for model generalization (avoid overfitting)
# 1 epoch is the full pass through the training data loader

dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = shuffle)

my_count = 0
for batch_inputs, batch_labels in dataloader: # iterate over a tuple
    my_count += 1
    print("="*80)
    print("my_count = ", my_count)
    print("batch_inputs: ", batch_inputs)
    print("batch_labels: ", batch_labels)



my_count =  1
batch_inputs:  tensor([[5.5000, 2.5000, 4.0000, 1.3000],
        [7.3000, 2.9000, 6.3000, 1.8000]], dtype=torch.float64)
batch_labels:  tensor([1, 2])
my_count =  2
batch_inputs:  tensor([[6.2000, 2.2000, 4.5000, 1.5000],
        [5.2000, 3.5000, 1.5000, 0.2000]], dtype=torch.float64)
batch_labels:  tensor([1, 0])
my_count =  3
batch_inputs:  tensor([[4.9000, 3.1000, 1.5000, 0.2000],
        [6.4000, 2.8000, 5.6000, 2.1000]], dtype=torch.float64)
batch_labels:  tensor([0, 2])
my_count =  4
batch_inputs:  tensor([[7.7000, 3.0000, 6.1000, 2.3000],
        [4.4000, 2.9000, 1.4000, 0.2000]], dtype=torch.float64)
batch_labels:  tensor([2, 0])
my_count =  5
batch_inputs:  tensor([[5.6000, 2.9000, 3.6000, 1.3000],
        [4.4000, 3.2000, 1.3000, 0.2000]], dtype=torch.float64)
batch_labels:  tensor([1, 0])
my_count =  6
batch_inputs:  tensor([[6.3000, 2.9000, 5.6000, 1.8000],
        [6.1000, 2.9000, 4.7000, 1.4000]], dtype=torch.float64)
batch_labels:  tensor([2, 1])
my_count =

## First Training Loop

Training Process So Far:
- Create a model
- Choose a loss function
- Define a dataset
- Set an optimizer
- Run a training loop
    - Calculate Loss (forward pass)
    - Compute Gradients (backpropogation)
    - Update model parameters

In [83]:
import seaborn as sns
import pandas as pd

# Load the mtcars dataset
df_mtcars = sns.load_dataset('mpg')
# Rename 'mpg' column to 'target' for consistency
df_mtcars = df_mtcars.rename(columns={'mpg': 'target'})

# Select feature columns (exclude target and non-numeric columns)
feature_cols = [col for col in df_mtcars.columns if col != 'target' and col != 'name' and df_mtcars[col].dtype in ['float64', 'int64']]

# Create features and target numpy arrays
features = df_mtcars[feature_cols]
X = df_mtcars[feature_cols].values
target = df_mtcars['target']
y = df_mtcars['target'].values

print(features.head(6))
# print(X)
print(target.head(6))
# print(y)

   cylinders  displacement  horsepower  weight  acceleration  model_year
0          8         307.0       130.0    3504          12.0          70
1          8         350.0       165.0    3693          11.5          70
2          8         318.0       150.0    3436          11.0          70
3          8         304.0       150.0    3433          12.0          70
4          8         302.0       140.0    3449          10.5          70
5          8         429.0       198.0    4341          10.0          70
0    18.0
1    15.0
2    18.0
3    16.0
4    17.0
5    15.0
Name: target, dtype: float64


In [84]:
dataset = TensorDataset(torch.tensor(X).float(), torch.tensor(y).float())
dataset

dataloader = DataLoader(dataset, batch_size = 4, shuffle = True)
dataloader

In [85]:
import torch
import torch.nn as nn
import torch.optim as optim

my_model = nn.Sequential(
    nn.Linear(6,2),
    nn.Linear(2,1)
)

# Create Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(my_model.parameters(), lr = 0.001) 

##### Training Loop

In [86]:
def show_results(model, dataloader):
    model.eval()
    iter_loader = iter(dataloader)
    for _ in range(3):
        print("="*80)
        feature, target = next(iter_loader)
        preds = model(feature)
        for p, t in zip(preds, target):
            print(f'Ground truth: {t.item():.3f}. Predicted: {p.item():.3f}.')

In [87]:
# todo: epoch is looping through entire dataset once
num_epochs = 10 # todo: how do we determine how many total epochs
for epoch in range(num_epochs):
    for data in dataloader:
        # set the gradients to zero
        optimizer.zero_grad()

        # get feature and target from data loader
        feature, target = data
        
        # Run a forward pass
        pred = my_model(feature)

        # Compute loss and gradient
        loss = criterion(pred, target)
        loss.backward()

        # update the parameters
        optimizer.step()
show_results(my_model, dataloader) # todo: fix this function, nan being printed

C:\Users\Delulu Lemon\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Delulu Lemon\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Ground truth: 34.100. Predicted: nan.
Ground truth: 22.000. Predicted: nan.
Ground truth: 15.000. Predicted: nan.
Ground truth: 24.000. Predicted: nan.
Ground truth: 36.000. Predicted: nan.
Ground truth: 24.500. Predicted: nan.
Ground truth: 21.500. Predicted: nan.
Ground truth: 26.000. Predicted: nan.
Ground truth: 20.500. Predicted: nan.
Ground truth: 18.500. Predicted: nan.
Ground truth: 10.000. Predicted: nan.
Ground truth: 13.000. Predicted: nan.


## ReLU Activation Functions